In [3]:
import numpy as np
import cv2
import smssend
import sys
from time import time

# 문자발송 설정
uid = "dhsvkf8456"
upw = "47f191289883b1f2f9417c344b42dd"

subject = "cctv 이상감지"
content = "사유지에 누군가 무단으로 침입하였습니다. cctv를 확인하세요."

hpno = "01050911251"
callback = "01050911251"

# HOG 모델 셋팅
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()
zone = []


#security_zone 범위 설정
def call_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        zone.append(x)
        zone.append(y)


switch1 = False
switch2 = True
switch3 = False

count = 0
count2 = 0

cap = cv2.VideoCapture('test.mp4')



while True:
    ret, frame = cap.read()
#     frame = cv2.resize(frame, (640, 480))
    frame = cv2.resize(frame, (520, 340))
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    boxes, weights = hog.detectMultiScale(frame,  winStride=(8,8))
    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])
    
        
        
    for (xA, yA, xB, yB) in boxes:
        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB),
                          (0, 255, 0), 2)

    #zone안에 일정시간 머물렀을 때 경고메세지 출력
    #security zone 범위안에 boxes가 있을 경우 switch 작동
    if len(boxes) == 1:
        for i in range(boxes[0][0], boxes[0][2]):
            if i <= zone[0] and i >= zone[2]:
                switch1 = False
            else:
                switch1 = True
        for i in range(boxes[0][1], boxes[0][3]):
            if i <= zone[1] and i >=zone[3]:
                switch1 = False
            else:
                switch1 = True
                
    #스위치가 작동되면 타이머에 현재 시간을 저장 
    #루프를 돌때마다 타이머가 리셋되는 걸 방지하기 위해 switch 하나 더 설정
    if switch1 == True and switch2 == True:
        timer = time()
        switch2 = False
     
    if switch1 == False and switch2 == False:  
        switch2 = True
   
    if switch1 == True and switch2 == False:
        count += time() - timer
        timer = time()
    
    if count > 30:
        frame = cv2.putText(frame, 'unidentified man detection!!', (50, 150),
                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0,220,200), 2, cv2.LINE_AA) 
        count2 = count
       
       
    if count2 > 30 and switch3 == False:
        count2 = 0
        jweb = smssend.JmunjaWeb(uid, upw)
        wresult = jweb.send(subject, content, hpno, callback)
        print('detection')
        switch3 = True    
          
# if cv2.waitKey(1) & 0xF == ord('r'):
#         count = 0
#         timer = time()
#         switch3 = False
        
    cv2.namedWindow('cctv', cv2.WINDOW_NORMAL)
    
    if len(zone) == 4:
        cv2.rectangle(frame, (zone[0],zone[1]), 
                      (zone[2],zone[3]), (0, 0, 255), 2)
    else:
        cv2.setMouseCallback('cctv', call_mouse, frame)
        
    cv2.imshow('cctv', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
        
        
cv2.destroyAllWindows()
cv2.waitKey()

detection


-1

In [ ]:
import smssend

In [ ]:
uid = "dhsvkf8456"
upw = "47f191289883b1f2f9417c344b42dd"
subject = "cctv 이상감지"


# 메세지 내용
content = "사유지에 누군가 무단으로 침입하였습니다. cctv를 확인하세요." 
# 전화번호
hpno = "01050911251"
callback = "01050911251"


# 메세지 전송
# jphone = smssend.JmunjaPhone(uid, upw)
# presult = jphone.send(subject, content, hpno)

# jweb = smssend.JmunjaWeb(uid, upw)
# wresult = jweb.send(subject, content, hpno, callback)

if presult or wresult:
    print("폰문자 %s건, 웹문자 %s건 발송 성공" % (presult, wresult))
else:
    print("발송 실패")